# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [97]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. '''

## 2. Generate the mathematical model

In [98]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [99]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model 

**Parameters:**

*  `s`: Syrup processing cost per hectolitre ($150) 
*  `p1`: Profit coefficient for 1.5 l COLI product ($49000)
*  `p2`: Profit coefficient for 1.5 l PEPSA product ($30)
*  `c`: Syrup conversion rate for COLI (40 units per hectolitre)
*  `d`: Syrup conversion rate for PEPSA (20 units per hectolitre)
*  `cap`: Hourly production capacity of S1 line (7100 units)
*  `h`: Daily working hours (8 hours)

**Decision Variables:**

*  `X1`: Number of 1.5 l COLI units produced daily
*  `X2`: Number of 1.5 l PEPSA units produced daily
*  `S`: Hectolitres of syrup used daily 

**Objective Function:**

Maximize daily profit:

```
Maximize Z = p1*X1 - X1^2 + p2*X2 - 2*X2^2 - s*S
```

**Constraints:**

1. **Syrup conversion:**
    *  `X1 <= c*S`
    *  `X2 <= d*S` 

2. **Production capacity:**
    *  `X1 + X2 <= cap*h` 

3. **Minimum PEPSA production:**
    *  `X2 >= 0.5*X1`

4. **Non-negativity:**
    *  `X1 >= 0`
    *  `X2 >= 0`
    *  `S >= 0`

This model aims to find the optimal daily production quantities of COLI (`X1`) and PEPSA (`X2`), along with the required syrup amount (`S`), that maximize the daily profit while respecting syrup conversion rates, production capacity, minimum PEPSA production target, and non-negativity constraints. 


## 3. Generate the pyomo code

In [100]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [101]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data 
s = 150  
p1 = 49000  
p2 = 30  
c = 40  
d = 20  
cap = 7100  
h = 8  

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)
model.S = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.profit = pyo.Objective(expr= p1*model.X1 - model.X1**2 + p2*model.X2 - 2*model.X2**2 - s*model.S, sense=pyo.maximize)

# Constraints
model.syrup_conversion1 = pyo.Constraint(expr= model.X1 <= c*model.S)
model.syrup_conversion2 = pyo.Constraint(expr= model.X2 <= d*model.S)

model.production_capacity = pyo.Constraint(expr= model.X1 + model.X2 <= cap*h)

model.min_pepsa = pyo.Constraint(expr= model.X2 >= 0.5*model.X1)

# Solve
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Display results
print(results)
print("X1:", pyo.value(model.X1))
print("X2:", pyo.value(model.X2))
print("S:", pyo.value(model.S))
print("Profit:", pyo.value(model.profit)) 
```

This code defines the model, variables, objective, and constraints as specified in your mathematical formulation. The data is input as sample values. You can replace these values with any other dataset you wish to use. Finally, the code calls the 'ipopt' solver to find the optimal solution and prints the results. 


## 4. Input problem data and try running the generated code

In [128]:
import pyomo.environ as pyo

# Sample data 
s = 150  
p1 = 49000  
p2 = 30  
c = 40  
d = 20  
cap = 7100  
h = 8  

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)
model.S = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.profit = pyo.Objective(expr= p1*model.X1 - model.X1**2 + p2*model.X2 - 2*model.X2**2 - s*model.S, sense=pyo.maximize)

# Constraints
model.syrup_conversion1 = pyo.Constraint(expr= model.X1 <= c*model.S)
model.syrup_conversion2 = pyo.Constraint(expr= model.X2 <= d*model.S)

model.production_capacity = pyo.Constraint(expr= model.X1 + model.X2 <= cap*h)

model.min_pepsa = pyo.Constraint(expr= model.X2 >= 0.5*model.X1)

# Solve
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Display results
print(results)
print("X1:", pyo.value(model.X1))
print("X2:", pyo.value(model.X2))
print("S:", pyo.value(model.S))
print("Profit:", pyo.value(model.profit)) 


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.05134391784667969
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

X1: 16337.083333340031
X2: 8168.541666660054
S: 408.4270833494343
Profit: 400350437.76073945


## 5. Correct the code to verify model viability (optional)

In [129]:
import pyomo.environ as pyo

# Sample data 
s = 150  
p1 = 49000  
p2 = 30  
c = 40  
d = 20  
cap = 7100  
h = 8  

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)
model.S = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.profit = pyo.Objective(expr= p1*model.X1 - model.X1**2 + p2*model.X2 - 2*model.X2**2 - s*model.S, sense=pyo.maximize)

# Constraints
model.syrup_conversion1 = pyo.Constraint(expr= model.X1/40 + model.X2/20 == model.S)

model.production_capacity = pyo.Constraint(expr= model.X1 + model.X2 <= cap*h)

model.min_pepsa = pyo.Constraint(expr= model.X2 >= 0.5*model.X1)

# Solve
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Display results
print(results)
print("X1:", pyo.value(model.X1))
print("X2:", pyo.value(model.X2))
print("S:", pyo.value(model.S))
print("Profit:", pyo.value(model.profit)) 


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.050195932388305664
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

X1: 16335.833333340031
X2: 8167.916666660053
S: 816.7916666665035
Profit: 400289176.04199195


## 6. Printing the outputs as strings, so they can be saved.

In [130]:
print(response.text)

## Mathematical Optimization Model 

**Parameters:**

*  `s`: Syrup processing cost per hectolitre ($150) 
*  `p1`: Profit coefficient for 1.5 l COLI product ($49000)
*  `p2`: Profit coefficient for 1.5 l PEPSA product ($30)
*  `c`: Syrup conversion rate for COLI (40 units per hectolitre)
*  `d`: Syrup conversion rate for PEPSA (20 units per hectolitre)
*  `cap`: Hourly production capacity of S1 line (7100 units)
*  `h`: Daily working hours (8 hours)

**Decision Variables:**

*  `X1`: Number of 1.5 l COLI units produced daily
*  `X2`: Number of 1.5 l PEPSA units produced daily
*  `S`: Hectolitres of syrup used daily 

**Objective Function:**

Maximize daily profit:

```
Maximize Z = p1*X1 - X1^2 + p2*X2 - 2*X2^2 - s*S
```

**Constraints:**

1. **Syrup conversion:**
    *  `X1 <= c*S`
    *  `X2 <= d*S` 

2. **Production capacity:**
    *  `X1 + X2 <= cap*h` 

3. **Minimum PEPSA production:**
    *  `X2 >= 0.5*X1`

4. **Non-negativity:**
    *  `X1 >= 0`
    *  `X2 >= 0`
    *  `S >= 0`

In [131]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data 
s = 150  
p1 = 49000  
p2 = 30  
c = 40  
d = 20  
cap = 7100  
h = 8  

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)
model.S = pyo.Var(domain=pyo.NonNegativeReals)

# Objective function
model.profit = pyo.Objective(expr= p1*model.X1 - model.X1**2 + p2*model.X2 - 2*model.X2**2 - s*model.S, sense=pyo.maximize)

# Constraints
model.syrup_conversion1 = pyo.Constraint(expr= model.X1 <= c*model.S)
model.syrup_conversion2 = pyo.Constraint(expr= model.X2 <= d*model.S)

model.production_capacity = pyo.Constraint(expr= model.X1 + model.X2 <= cap*h)

model.min_pepsa = pyo.Constraint(expr= model.X2 >= 0.5*model.X1)

# Solve
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Display results
print(results)
print("X1:", pyo.value(model.X1))
print("X2:", pyo.value(